In [1]:
import os

os.environ["TAVILY_API_KEY"] = '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'
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = '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'
os.environ["HTTP_PROXY"] = '192.168.208.104:7890'
os.environ["HTTPS_PROXY"] = '192.168.208.104:7890'

In [2]:
from langchain_community.embeddings import HuggingFaceEmbeddings

hf = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en", model_kwargs={"device": "cpu"})

/home/zhuxy/anaconda3/envs/OSAF/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/zhuxy/anaconda3/envs/OSAF/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/home/zhuxy/anaconda3/envs/OSAF/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If

In [3]:
os.environ["HTTP_PROXY"] = ''
os.environ["HTTPS_PROXY"] = ''
os.environ["DASHSCOPE_API_KEY"]='sk-c1bb7b06f7434e2aa6871f22f088cd78'

In [4]:
from langchain_community.vectorstores import Chroma
# vectordb = Chroma.from_documents(
#     documents=all_contents,
#     collection_name='seL4_md',
#     embedding=hf,
#     persist_directory="../vectordb"
# )
vectordb = Chroma(
    collection_name='seL4_md',
    embedding_function=hf,
    persist_directory="../vectordb"
)
retriever = vectordb.as_retriever(search_kwargs={"k": 7})

In [5]:
### Retrieval Grader

run_local = "Yes"
local_llm = "llama3.1:70b"

from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser,StrOutputParser

# LLM

llm = ChatOllama(model=local_llm, temperature=0.5, timeout=120, num_ctx=12288)
json_llm = ChatOllama(model=local_llm, format="json", temperature=0.5, timeout=120, num_ctx=12288)

grader_prompt = PromptTemplate(
    template="""You are a grader assessing relevance of a retrieved document to a user question. \n 
    Here is the retrieved document: \n\n {document} \n\n
    Here is the user question: I want to formalize seL4, and the first step is to write specifications based on properties.\n
    Please try to provide some properties of {question}. \n
    If the document is useful for answering the question, or provide the relevent Isabelle code, grade it as useful. \n
    If the document is relevant but not useful, or not relevant at all, grade it as not useful. \n
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is useful to the question. \n
    Provide the binary score as a JSON with a single key 'score' and no premable or explanation.""",
    input_variables=["question", "document"],
)

retrieval_grader = grader_prompt | json_llm | JsonOutputParser()
question = "Kenel Heap Access"
docs = retriever.get_relevant_documents(question)
doc_txt = docs[0].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))
print(doc_txt)

/home/zhuxy/anaconda3/envs/OSAF/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


KeyboardInterrupt: 

In [6]:
# Distiller

distill_prompt = PromptTemplate(
    template="""You are an assistant specialized in information distillation. \n 
    You are given a document and a question.
    Here is the document: \n\n {document} \n\n
    Here is the question: Please try to provide some properties of {question}. \n
    Here is what you need to do:
    Please distill the document while retaining all information that is useful for answering the question.
    Please answer in concise language while ensuring that as little useful information as possible is lost.
    Provide the answer with and only with the distilled documents,no openning or closing statement.""",
    input_variables=["question", "document"],
)

distiller = distill_prompt | llm | StrOutputParser()
question = "IPC"
docs = retriever.get_relevant_documents(question)
doc_txt = docs[1].page_content
# try:
#     print(distiller.invoke({"question": question, "document": doc_txt}))
# except Exception as e:
#     print("TimeoutError")
# print("---")
# print(doc_txt)

In [7]:
from langchain_core.output_parsers import StrOutputParser
### Question Re-writer

# LLM

# Prompt
re_write_prompt = PromptTemplate(
    template="""You a question re-writer that converts an input question to a better version that is optimized \n 
     for vectorstore retrieval. Look at the initial and formulate an improved question. \n
     Here is the initial question: \n\n I want to formalize the IPC of seL4, and the first step is to write specifications based on properties.\n
    Please try to provide some properties of {question}. Improved question with no preamble, answer with the improved question only: \n """,
    input_variables=["generation", "question"],
)

question_rewriter = re_write_prompt | llm | StrOutputParser()
# question_rewriter.invoke({"question": 'Please try to find relevent content of the following property of seL4:'+question})

In [8]:
examples = [
{
"concept": "Activating Threads",
"context": """
Threads that are active always have a master Reply capability to themselves stored in their reply slot.
This is so that a derived Reply capability can be generated immediately if they wish to issue one.
This function sets up a new master Reply capability if one does not exist.
definition
"setup_reply_master thread ≡ do
old_cap <- get_cap (thread, tcb_cnode_index 2);
when (old_cap = NullCap) $ do
set_original (thread, tcb_cnode_index 2) True;
set_cap (ReplyCap thread True) (thread, tcb_cnode_index 2)
od
od"

Reactivate a thread if it is not already running.
definition
restart :: "obj_ref ⇒ (unit,’z::state_ext) s_monad" where
"restart thread ≡ do
state ← get_thread_state thread;
when (¬ runnable state ∧ ¬ idle state) $ do
ipc_cancel thread;
setup_reply_master thread;
set_thread_state thread Restart;
do_extended_op (tcb_sched_action (tcb_sched_enqueue) thread);
do_extended_op (switch_if_required_to thread)
od
od"
""",
"answer":"""
Reply Capability Setup: Active threads have a master Reply capability stored in their reply slot. This is so that a derived Reply capability can be generated immediately if they wish to issue one. If a master Reply capability does not exist, the function sets up a new one.
Thread Reactivation: Reactivates a thread if it is not already running by canceling IPC, setting up a master Reply capability, setting the thread state to Restart, and re-enqueuing the thread with the scheduler.
"""
},
{
"concept": "",
"context": """
definition
get_message_info :: "obj_ref ⇒ (message_info,’z::state_ext) s_monad"
where
"get_message_info thread ≡ do
x ← as_user thread $ get_register msg_info_register;
return $ data_to_message_info x
od"
""",
"answer":"""
retrieves the message information of a thread by accessing the message info register and converting the data into a message info structure."""
},
{
"concept": "Generic functions on top of the state monad",
"context": """
Apply a function to the current state and return the result without changing the state.
definition
gets :: "(’s ⇒ ’a) ⇒ (’s, ’a) nondet_monad" where
"gets f ≡ get >>= (λs. return (f s))"
Modify the current state using the function passed in.
definition
modify :: "(’s ⇒ ’s) ⇒ (’s, unit) nondet_monad" where
"modify f ≡ get >>= (λs. put (f s))"
lemma simpler_gets_def: "gets f = (λs. ({{(f s, s)}}, False))"
lemma simpler_modify_def:
"modify f = (λs. ({{((), f s)}}, False))"
Execute the given monad when the condition is true, return () otherwise.
definition
when :: "bool ⇒ (’s, unit) nondet_monad ⇒
(’s, unit) nondet_monad" where
"when P m ≡ if P then m else return ()"
Execute the given monad unless the condition is true, return () otherwise.
definition
unless :: "bool ⇒ (’s, unit) nondet_monad ⇒
(’s, unit) nondet_monad" where
"unless P m ≡ when (¬P) m"
Perform a test on the current state, performing the left monad if the result is true or the right monad
if the result is false.
definition
condition :: "(’s ⇒ bool)
⇒ (’s, ’r) nondet_monad
⇒ (’s, ’r) nondet_monad
⇒ (’s, ’r) nondet_monad"
where
"(condition P L R) ≡ λs. if (P s) then (L s) else (R s)"
notation (output)
condition ("(condition (_)// (_)// (_))" [1000,1000,1000] 1000)
Apply an option valued function to the current state, fail if it returns None, return v if it returns Some
v.
definition
gets_the :: "(’s ⇒ ’a option) ⇒ (’s, ’a) nondet_monad" where
"gets_the f ≡ gets f >>= assert_opt"
""",
"answer":"""
Applying Functions to State: This involves applying a function to the current state to retrieve a result without changing the state. It enables querying the state in a nondestructive manner.
Modifying State: This operation modifies the current state using a provided function. It ensures that the state can be updated and maintained accurately within the monad.
Conditional Execution: These operations execute monads based on specified conditions. They facilitate branching logic within computations, allowing for different actions based on the state or provided conditions.
State-Dependent Execution: This involves performing tests on the current state to decide which monad to execute. It ensures that computations can adapt based on the state, allowing for flexible and context-aware operations.
Applying Optional Functions to State: This operation applies functions that return optional values to the state. It ensures that computations only proceed with valid data, handling cases where data might be missing.
"""
},
{
"concept": "Abstract Specification Instantiations",
"context": """
theory Deterministic_A
imports
Structures_A
"../../lib/List_Lib"
begin
The kernel specification operates over states of type ’a state, which includes all of the abstract
kernel state plus an extra field, exst of type ’a. By choosing an appropriate concrete type for ’a,
we obtain different instantiations of this specification, at differing levels of abstraction. The abstract
specification is thus extensible. The basic technique, and its motivation, are described in [14].
Here, we define two such instantiations. The first yields a largely-deterministic specification by instantiating ’a with a record that includes concrete scheduler state and information about sibling ordering
in the capability derivation tree (CDT). We call the resulting specification the deterministic abstract
specification and it is defined below in section 19.1.
The second instantiation uses the type unit for ’a, yielding a specification that is far more nondeterministic. In particular, the scheduling behaviour and the order in which capabilities are deleted during
a revoke system call both become completely nondeterministic. We call this second instantiation the
nondeterministic abstract specification and it is defined below in section 19.2.
Translate a state of type ’a state to one of type ’b state via a function t from ’a to ’b.
definition trans_state :: "(’a ⇒ ’b) ⇒ ’a state ⇒ ’b state" where
"trans_state t s = (|kheap = kheap s, cdt = cdt s, is_original_cap = is_original_cap s,
cur_thread = cur_thread s, idle_thread = idle_thread s,
machine_state = machine_state s,
interrupt_irq_node = interrupt_irq_node s,
interrupt_states = interrupt_states s, arch_state = arch_state s,
exst = t(exst s)|)"
Truncate an extended state of type ’a state by effectively throwing away all the ’a information.
abbreviation "truncate_state ≡ trans_state (λ_. ())"
""",
"answer":"""
Deterministic Abstract Specification: The kernel specification operates over states of type 'a state, which includes all of the abstract kernel state plus an extra field, exst of type 'a. By choosing an appropriate concrete type for 'a, different instantiations of the specification at varying levels of abstraction are obtained. This specification results from instantiating 'a with a record that includes concrete scheduler state and information about sibling ordering in the capability derivation tree (CDT), yielding a largely-deterministic specification.
Nondeterministic Abstract Specification: This specification results from using the type unit for 'a, yielding a specification with highly nondeterministic behavior, particularly in scheduling and capability deletion order during a revoke system call.
State Transformation: Transforms the state by applying a given transformation function to the extension part of the state while keeping all other parts unchanged.
State Truncation: Transformation of the state by truncating the extension part to a unit value, effectively discarding any extra information while keeping all other parts of the state unchanged.
"""
},
{
"concept":"",
"context": """
Capturing the protection state in such a policy simplifies reasoning in three ways. First,the number of labels is typically smaller than the number of objects in the system-depending on the system,a label can contain thousands of objects.Second, when using the policy to reason about untrusted subjects,the policy will typically be static over each system call whereas the protection state will typically change.Finally, we can formulate the allowed effects of operations of untrusted subjects by consulting the policy rather than the more complex protection state. 
The access rights between labels in our model are:Receive,SyncSend,AsyncSend, Reset,Grant,Write,Read,and Control.We make a distinction between synchronous and asynchronous send,because the former has different implications for confidential- ity as explained later in Section 5.3.The two other nonstandard rights are Reset and Control.The former is the authority to reset an object to its initial state for re-using resources.The latter confers the authority to completely determine the behaviour of the object,for example,by writing to a thread's registers.The Create right known from the classical take-grant models [Lipton and Snyder 1977]is subsumed here by Control.
""",
"answer":"""
Access Control Policy: The policy simplifies reasoning by using labels instead of objects, which are fewer in number. This static policy helps in reasoning about untrusted subjects, whose protection state changes dynamically.
Access Rights Between Labels: The access rights include Receive, SyncSend, AsyncSend, Reset, Grant, Write, Read, and Control. Synchronous and asynchronous send are distinguished due to their different confidentiality implications. Reset allows resetting an object to its initial state, while Control grants authority to determine the behavior of an object. The Create right is subsumed by Control in this model.
"""
},
{
"concept": "Non-Interference",
"context": """More well-defined than general information flow is the property of non-interference (Goguen and Meseguer, 1982, 1984; Rushby, 1992). This property is able to capture indirect information flows in a model if the code for both sides is known. Given a secret process S and an attacker A, it requires one to prove that the behaviour of A cannot depend on the actions of S, i.e. that S does not interfere with the behaviour of A. It is possible to establish this property by looking at execution traces of the system: Given a trace where execution of both S and A occurs, one needs to show that the result in A is the same when all S actions are removed from the trace. Unfortunately, this property is not necessarily preserved by the proof technique of refinement which is the one most commonly used to relate an implementation to its model (Jacob, 1989). This means that proving non-interference of the model only implies non-interference of the implementation when special care is taken in the implementation proof. The standard access control models do not have this problem.""",
"answer": """Non-Interference: Capture indirect information flows in a model when the code for both the secret process (S) and the attacker (A) is known. It requires proving that the behavior of A does not depend on the actions of S, meaning S does not interfere with A. This can be established by examining execution traces and showing that the result in A remains the same when all S actions are removed."""
}
]



In [9]:
# examples = [
#     {
#         "concept": "Accessing the Kernel Heap",
#         "context": """[
# 'The kernel’s heap is a partial function containing kernel objects. The type synonym for the kernel heap is kheap = obj_ref ⇒ kernel_object option.',
# 'Kernel memory management in seL4 is explicit. All kernel data structures are either statically allocated at boot time or are dynamically-allocated first-class objects in the API.',
# 'The kernel’s memory allocation model has the following core characteristics: Memory allocation is explicit, only performed on retyping an untyped memory object, and strictly bounded by available memory in an untyped memory object.',
# 'Kernel Heap Accessors: Manipulate ASID pools, page directories, and page tables in the kernel heap. The functions associated with this include get_asid_pool, which takes an obj_ref parameter and returns an ASID pool from the kernel heap, and set_asid_pool, which takes an obj_ref and an ASID pool, setting the ASID pool in the kernel heap.',
# 'ASID High Bits: The high bits of a virtual ASID are defined by the function asid_high_bits_of, which maps an ASID to a word8 value using ucast asid >> asid_low_bits.'
#     ]""",
#         "answer": """Memory Allocation Boundaries: Ensure that memory allocation in the kernel heap is strictly bounded by the available memory in an untyped memory object.
# Explicit Memory Management: Specify that all kernel heap allocations must be explicitly requested and managed.
# Partial Function Integrity: Ensure that the kernel heap remains a partial function, mapping object references to kernel objects without any invalid references.
# ASID Pool Management: Ensure that ASID pools can be correctly retrieved and set within the kernel heap.
# Static and Dynamic Allocation Separation: Distinguish between statically allocated data structures and dynamically allocated objects in the kernel heap."""
#     },
#     # {
#     #     "concept": "Non-Interference",
#     #     "context": """[More well-defined than general information flow is the property of non-interference (Goguen and Meseguer, 1982, 1984; Rushby, 1992). This property is able to capture indirect information flows in a model if the code for both sides is known. Given a secret process S and an attacker A, it requires one to prove that the behaviour of A cannot depend on the actions of S, i.e. that S does not interfere with the behaviour of A. It is possible to establish this property by looking at execution traces of the system: Given a trace where execution of both S and A occurs, one needs to show that the result in A is the same when all S actions are removed from the trace. Unfortunately, this property is not necessarily preserved by the proof technique of refinement which is the one most commonly used to relate an implementation to its model (Jacob, 1989). This means that proving non-interference of the model only implies non-interference of the implementation when special care is taken in the implementation proof. The standard access control models do not have this problem.]""",
#     #     "answer": """Non-interference: Given a secret process S and an attacker A, non-interference requires that A's behavior cannot depend on the actions of S. This property can be established by examining the execution traces of a system. Given a trace containing actions of both S and A, it must be proven that if all actions of S are removed from the trace, the outcome for A remains unchanged."""
#     # },
#     {
#         "concept": "Abstract Address Space Model",
#         "context": """[We have shown a number of safety properties about the abstract address space model. They are formulated as invariants over the abstract datatype. A set of states I is an invariant if it contains all initial states and if execution of any operation in a state of I again leads to a state in I. We write D |= I when I is an invariant of data type D. Theorem 1. There are no loops in the address space structure. A |= {{s | ∀ x . ¬ s ` x ❀+ x}} The proof is by case distinction on the operations and proceeds by observing how each operation changes existing paths. Theorem 1 is significant for implementing the lookup function efficiently. It also ensures that internal kernel functions can walk the corresponding data structures naively. Together with the properties below it says that address spaces always have a tree structure. Theorem 2. All valid pages translate to physical pages. A |= {{s | ∀ x . s ` x −→ (∃ r . s ` x ❀∗ Real r )}} The proof is again by case distinction on the operations. Together with the following theorem we obtain that address lookup is a total function on data type A. Theorem 3. The lookup relation is a function. [[s ` n,v ✄ r ; s ` n,v ✄ r 0 ]] =⇒ r = r 0 This theorem follows directly from the fact that paths are built on functions. That address lookup is a total function may sound like merely a nice formal property, but it is quite literally an important safety property in reality. Undefined behavior, possibly physical damage, may result if two conflicting TLB entries are present for the same virtual address. The current ARM reference manual [1, p. B3-26] explicitly warns against this scenario.]""",
#         "answer": """
# No loops in the address space structure: Proven by case distinction on operations.Important for efficient lookup function implementation and naive traversal of data structures by kernel functions.
# All valid pages map to physical pages: Proven by case distinction on operations.
# The lookup relation is a function: Proven directly from the construction of paths on functions."""
#     }
# ]


In [9]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # This is the list of examples available to select from.
    examples,
    # This is the embedding class used to produce embeddings which are used to measure semantic similarity.
    hf,
    # This is the VectorStore class that is used to store the embeddings and do a similarity search over.
    Chroma,
    # This is the number of examples to produce.
    k=2
)


example_prompt = PromptTemplate(input_variables=["concept", "context",'answer'], template="Concept:\n{concept}\nContext:\n{context}\nAnswer:\n{answer}")

generation_prompt = FewShotPromptTemplate(
    example_prompt=example_prompt, 
    example_selector=example_selector,
    prefix="""Please help me with formalizing the seL4 OS by writing specifications based on its properties and behavior. Here is examples:""", 
    suffix='Now, based on the provided context below, suggest properties to specify. Limit suggestions to what can be directly inferred from the context; avoid unnecessary assumptions.  \n Concept:\n{concept}\n Context:\n{context}\n Answer:',
    input_variables=["concept",'context']
)

merge_prompt = '''
Merge and refine the suggested properties into concise tips.
Combine similar or related properties to eliminate duplication.
Ensure each tip maintains the same level of detail.
Respond in JSON format as follows:
{
"specifications": [
  "Name: Detail",
  "Name: Detail",
  ...
]
}
Do not include any additional text outside of this JSON structure.
'''

# Post-processing
def format_docs(docs):
    joined_docs = []
    for doc in docs:
        content = doc.metadata['content']
        paragraph = doc.metadata['paragraph']
        section = doc.metadata['section'] 
        joined_docs.append(f"Paragraph:{paragraph} - Section:{section}\n{content}")
    
    return '\n\n'.join(joined_docs)

In [10]:
# ### Generate
from langchain_community.chat_models import ChatOllama
from langchain.schema.output_parser import StrOutputParser
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

# memory = ConversationBufferMemory()
# conversation = ConversationChain(
#     llm=json_llm,
#     memory=memory,
# )

# concept = 'Parent:ARM-Specific Data Types.Current Concept to specify:Architecture-specific state'

# context = retriever.invoke(concept)
# context = format_docs(context)
# # Run
# response1 = conversation.predict(input=generation_prompt.format(context=context, concept=concept))


# response2 = conversation.predict(input=merge_prompt)

# print(response2)

In [11]:
from typing import List

from typing_extensions import TypedDict


class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        web_search: whether to add search
        documents: list of documents
    """

    question: str
    generation: str
    web_search: str
    documents: List[str]
    better_question: str
    web_documents: List[str]

In [12]:
### Search

from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults(max_results=4)

In [18]:
# 本地

from langchain.schema import Document



def retrieve(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = retriever.get_relevant_documents(question)
    # doc_content = []
    # for d in documents:
    #     doc_content.append(d.page_content)
    return {"documents": documents, "question": question}


def distill(state):
    print("---DISTILL---")
    question = state["question"]
    documents = state["documents"]
    web_documents = state["web_documents"]

    if web_documents:
        for d in web_documents:
            documents.append(d)

    return {"documents": documents, "question": question}
    
    # #RAG generation
    # distill_docs = []
    # for d in documents:
    #     if len(d.page_content)<200:
    #         distill_docs.append(d.page_content)
    #     else:
    #         try_count = 0
    #         max_tries = 3  # 可以设置最大重试次数
    #         while try_count < max_tries:
    #             try:
    #                 distill_doc = distiller.invoke({"document": d, "question": question})
    #                 distill_docs.append(distill_doc)
    #                 break
    #             except Exception as e:
    #                 try_count += 1
            
    # return {"documents": distill_docs, "question": question}

def generate(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]

    context = format_docs(documents)

    # RAG generation
    try_count = 0
    max_tries = 3  # 可以设置最大重试次数
    while try_count < max_tries:
        try:
            memory = ConversationBufferMemory()
            conversation = ConversationChain(
                llm=json_llm,
                memory=memory,
            )
            # Run
            conversation.predict(input=generation_prompt.format(context=context, concept=question))
            generation = conversation.predict(input=merge_prompt)
            break
        except Exception as e:
            try_count += 1
            
    return {"documents": documents, "question": question, "generation": generation}


def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with only filtered relevant documents
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    
    question = state["question"]
    if state["web_documents"]:
        documents = state["web_documents"]
    else:
        documents = state["documents"]

    # Score each doc
    filtered_docs = []
    # web_search = "No"
    for d in documents:
        score = retrieval_grader.invoke(
            {"question": question, "document": d}
        )
        grade = score["score"]
        if grade == "yes":
            print("---GRADE: DOCUMENT USEFUL---")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT USEFUL---")
            # web_search = "Yes"
            continue
    return {"documents": filtered_docs, "question": question}

def transform_query(state):
    """
    Transform the query to produce a better question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates question key with a re-phrased question
    """

    print("---TRANSFORM QUERY---")
    if state['better_question']:
        question = state['better_question']
    else:
        question = state["question"]
    documents = state["documents"]

    # Re-write question
    better_question = question_rewriter.invoke({"question": question})
    return {"documents": documents, "better_question": better_question}


def web_search(state):
    """
    Web search based on the re-phrased question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with appended web results
    """
    return
    # print("---WEB SEARCH---")
    
    # question = state["better_question"]
    # web_documents = []

    # Web search
    # docs = web_search_tool.invoke({"query": question})
    # print(docs)
    # for d in docs:
    #     web_documents.append(d["content"])

    #return {"web_documents": web_documents}


### Edges


def decide_to_generate(state):
    """
    Determines whether to generate an answer, or re-generate a question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    # state["question"]
    # web_search = state["web_search"]
    len_doc = len(state["documents"]) if state["documents"] else 0

    print(len_doc)
    # if web_search == "Yes":
    if len_doc<2:
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print(
            "---DECISION: NEED MORE DOC, TRANSFORM QUERY---"
        )
        return "transform_query"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: DISTILL AND GENERATE---")
        return "distill"

In [13]:
# 调用API,用这部分代码

from langchain.schema import Document
import os
from openai import OpenAI
from random import randint



def get_response(messages):
    key = ['sk-a6c628b29e2f4b65b0ecd322f21e669f','sk-f4ebc969b87e41d7bf1dc91dd5026bba','sk-0fc3a02ca767436d91bea234f2f9d4e7','sk-de00e7083c4542458bfdcdc8158374a7','sk-68921001062244bd9d71139b015d1b9c']

    with open('./flag.txt','r') as file:
        i = int(file.read().strip())
    
    with open('./flag.txt','w') as file:
        file.write(str(i+1))

    client = OpenAI(
        api_key=key[i%5],
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1", 
    )
    completion = client.chat.completions.create(model="llama3.1-405b-instruct", messages=messages)
    return completion.choices[0].message.content



def retrieve(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = retriever.get_relevant_documents(question)
    # doc_content = []
    # for d in documents:
    #     doc_content.append(d.page_content)
    return {"documents": documents, "question": question}


def distill(state):
    print("---DISTILL---")
    question = state["question"]
    documents = state["documents"]
    web_documents = state["web_documents"]

    if web_documents:
        for d in web_documents:
            documents.append(d)

    return {"documents": documents, "question": question}
    
    # #RAG generation
    # distill_docs = []
    # for d in documents:
    #     if len(d.page_content)<200:
    #         distill_docs.append(d.page_content)
    #     else:
    #         try_count = 0
    #         max_tries = 3  # 可以设置最大重试次数
    #         while try_count < max_tries:
    #             try:
    #                 distill_doc = distiller.invoke({"document": d, "question": question})
    #                 distill_docs.append(distill_doc)
    #                 break
    #             except Exception as e:
    #                 try_count += 1
            
    # return {"documents": distill_docs, "question": question}

def generate(state):
    """
    Generate answer using API call and complete two rounds of dialogue

    Args:
        state (dict): The current graph state
        generation_prompt (str): Prompt for generating the initial response
        merge_prompt (str): Prompt for merging the context and question

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]
    context = format_docs(documents)
    # Prepare messages for the API call
    messages = [
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": generation_prompt.format(context=context, concept=question),
        }
    ]

    try_count = 0
    max_tries = 3  # 可以设置最大重试次数
    while try_count < max_tries:
        try:
            # First round of dialogue
            first_generation = get_response(messages)
            
            assert first_generation
            print(first_generation)
            # Add the first generation to the messages list
            messages.append({"role": "assistant", "content": first_generation})
            
            # Second round of dialogue with merge prompt
            messages.append({"role": "user", "content": merge_prompt})
            second_generation = get_response(messages)
            
            break
        except Exception as e:
            try_count += 1
            
    return {"documents": documents, "question": question, "generation": second_generation}


def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with only filtered relevant documents
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    
    question = state["question"]
    if state["web_documents"]:
        documents = state["web_documents"]
    else:
        documents = state["documents"]

    # Score each doc
    filtered_docs = []
    # web_search = "No"
    for d in documents:
        score = retrieval_grader.invoke(
            {"question": question, "document": d}
        )
        grade = score["score"]
        if grade == "yes":
            print("---GRADE: DOCUMENT USEFUL---")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT USEFUL---")
            # web_search = "Yes"
            continue
    return {"documents": filtered_docs, "question": question}

def transform_query(state):
    """
    Transform the query to produce a better question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates question key with a re-phrased question
    """

    print("---TRANSFORM QUERY---")
    if state['better_question']:
        question = state['better_question']
    else:
        question = state["question"]
    documents = state["documents"]

    # Re-write question
    better_question = question_rewriter.invoke({"question": question})
    return {"documents": documents, "better_question": better_question}


def web_search(state):
    """
    Web search based on the re-phrased question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with appended web results
    """
    return
    # print("---WEB SEARCH---")
    
    # question = state["better_question"]
    # web_documents = []

    # Web search
    # docs = web_search_tool.invoke({"query": question})
    # print(docs)
    # for d in docs:
    #     web_documents.append(d["content"])

    #return {"web_documents": web_documents}


### Edges


def decide_to_generate(state):
    """
    Determines whether to generate an answer, or re-generate a question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    # state["question"]
    # web_search = state["web_search"]
    len_doc = len(state["documents"]) if state["documents"] else 0

    print(len_doc)
    # if web_search == "Yes":
    if len_doc<2:
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print(
            "---DECISION: NEED MORE DOC, TRANSFORM QUERY---"
        )
        return "transform_query"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: DISTILL AND GENERATE---")
        return "distill"

In [14]:
from langgraph.graph import END, StateGraph

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("grade_documents", grade_documents)  # grade documents
workflow.add_node("generate", generate)  # generatae
workflow.add_node("transform_query", transform_query)  # transform_query
workflow.add_node("web_search_node", web_search)  # web search
workflow.add_node("distill", distill)
workflow.add_node("grade_websearch", grade_documents)

# Build graph
workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "transform_query": "transform_query",
        "distill": "distill",
    },
)
workflow.add_edge("transform_query", "web_search_node")
# workflow.add_edge("web_search_node", "generate")
workflow.add_edge("web_search_node", "grade_websearch")
workflow.add_edge("grade_websearch", "distill")
workflow.add_edge("distill","generate")
workflow.add_edge("generate", END)

# Compile
app = workflow.compile()

In [15]:
import os
from neo4j import GraphDatabase

# 从环境变量加载Neo4j连接信息
NEO4J_URI = os.getenv("NEO4J_URI", "bolt://localhost:7687")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME", "neo4j")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD", "neo4jneo4j")

# 创建Neo4j驱动
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

def write_queries_to_neo4j(query, valid_queries):
    def add_query_and_rel(tx, query, sub_query):
        tx.run("""
            MERGE (q:Query {name: $query})
            MERGE (sq:Query {name: $sub_query})
            MERGE (q)-[:SPECIFY]->(sq)
            """, parameters={"query": query, "sub_query": sub_query})
    
    with driver.session() as session:
        for sub_query in valid_queries:
            # 跳过自己指向自己
            if query == sub_query:
                continue
            session.write_transaction(add_query_and_rel, query, sub_query)

In [16]:
import os
from neo4j import GraphDatabase

# 从环境变量加载Neo4j连接信息
NEO4J_URI = os.getenv("NEO4J_URI", "bolt://localhost:7687")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME", "neo4j")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD", "neo4jneo4j")

# 创建Neo4j驱动
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# 定义查询语句，查找所有叶子节点（无出度）及其父节点
query = """
MATCH (parent:Query)-[:SPECIFY]->(n:Query)
WHERE NOT (n)-->()
RETURN parent.name AS parent_name, n.name AS leaf_node_name
"""

log_name = {}

# 执行查询并获取结果
def get_leaf_nodes_with_parents(tx):
    result = tx.run(query)

    queries = []
    for record in result:
        q = f"Parent: {record['parent_name']}. Current Concept to specify: {record['leaf_node_name']}"
        if record['parent_name']=='seL4':
            log_name[q]=record['leaf_node_name']
        elif record['parent_name'] in ['The Monad','Adding Exceptions','Catching and Handling Exceptions','Hoare Logic','Syntax']:
            log_name[q]='Nondeterministic State Monad with Failure'
        else:
            log_name[q]=record['parent_name']
        queries.append(q)
    
    return queries

with driver.session() as session:
    leaf_nodes_with_parents = session.read_transaction(get_leaf_nodes_with_parents)
    print("Leaf Nodes with Parents:", leaf_nodes_with_parents)

# 关闭驱动
driver.close()

Leaf Nodes with Parents: ['Parent: The Monad. Current Concept to specify: Nondeterminism', 'Parent: The Monad. Current Concept to specify: Failure', 'Parent: The Monad. Current Concept to specify: Generic functions on top of the state monad', 'Parent: The Monad. Current Concept to specify: The Monad Laws', 'Parent: Adding Exceptions. Current Concept to specify: Monad Laws for the Exception Monad', 'Parent: Syntax. Current Concept to specify: Syntax for the Nondeterministic State Monad', 'Parent: Syntax. Current Concept to specify: Syntax for the Exception Monad', 'Parent: Nondeterministic State Monad with Failure. Current Concept to specify: Library of Monadic Functions and Combinators', 'Parent: Catching and Handling Exceptions. Current Concept to specify: Loops', 'Parent: Hoare Logic. Current Concept to specify: Validity', 'Parent: Hoare Logic. Current Concept to specify: Determinism', 'Parent: Hoare Logic. Current Concept to specify: Non-Failure', 'Parent: Nondeterministic State Mon

/tmp/ipykernel_604890/2900019317.py:39: DeprecationWarning: read_transaction has been renamed to execute_read
  leaf_nodes_with_parents = session.read_transaction(get_leaf_nodes_with_parents)


In [17]:
len(leaf_nodes_with_parents)

91

In [43]:
leaf_nodes_with_parents[40:]

['Parent: Accessing the Kernel Heap. Current Concept to specify: General Object Access',
 'Parent: Accessing the Kernel Heap. Current Concept to specify: TCBs',
 'Parent: Accessing the Kernel Heap. Current Concept to specify: Synchronous and Asyncronous Endpoints',
 'Parent: Accessing the Kernel Heap. Current Concept to specify: IRQ State and Slot',
 'Parent: Accessing the Kernel Heap. Current Concept to specify: User Context',
 'Parent: Accessing CSpace. Current Concept to specify: Capability access',
 'Parent: Accessing CSpace. Current Concept to specify: Accessing the capability derivation tree',
 'Parent: Accessing the ARM VSpace. Current Concept to specify: Encodings',
 'Parent: Accessing the ARM VSpace. Current Concept to specify: Kernel Heap Accessors',
 'Parent: Accessing the ARM VSpace. Current Concept to specify: Basic Operations',
 'Parent: seL4. Current Concept to specify: ARM Object Invocations',
 'Parent: seL4. Current Concept to specify: Kernel Object Invocations',
 'Par

In [19]:
import time
from pprint import pprint
from langchain_community.chat_models import ChatOllama
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
import re

queries = leaf_nodes_with_parents[1:]

# 获取当前时间戳的后几位作为文件名的一部分
timestamp_suffix = str(int(time.time()))[-6:]
prefix = 'test'
#output_file = f"./output/{prefix}_{timestamp_suffix}.txt"

def recursive_rag_call(query, depth=0, max_depth=1):   
    
    # 如果达到递归深度或score达到阈值，停止递归
    if depth >= max_depth:
        return
    
    # 调用 RAG 链
    inputs = {"question": query}
    for output in app.stream(inputs):
        for key, value in output.items():
            # Node
            pprint(f"Node '{key}':")
            # Optional: print full state at each node
            # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
        pprint("\n---\n")

    # Final generation
    result = value["generation"]
    #pprint(result)
    
    
    # 获取新的子 query（假设 result 中包含子 query 列表）
    new_queries = extract_sub_queries(result)  # 提取子 query

    if len(new_queries)<=1:
        return
    
    valid_queries = []
    # 对每个新的子 query 进行递归调用
    for new_query in new_queries:
        if new_query:
            valid_queries.append(new_query)
        # 写入output文件
    with open(f"./KG/llama405b/{log_name[query]}.txt", "a") as f:
        f.write(f"Depth: {depth}, Query:\n{query}\n")
        f.write(f"Result:\n")        
        for valid_query in valid_queries:
            f.write(f"{valid_query}\n")


    # 正则表达式匹配模式
    pattern = r"Current Concept to specify: (?P<concept>.+)$"

    # 使用正则表达式提取 leaf 部分
    match = re.search(pattern, query)

    if match:
        query = match.group("concept")
        
    write_queries_to_neo4j(query, valid_queries)

    for valid_query in valid_queries:
        recursive_rag_call(valid_query, depth + 1, max_depth)

def extract_sub_queries(result):
    try:
        # 尝试将 JSON 字符串解析为字典
        import json
        data = json.loads(result)
        # 返回字典中的 specifications 列表
        return data.get("specifications", [])
    except json.JSONDecodeError:
        # 如果解析失败，则返回一个空列表或处理错误
        return []

for query in queries:
    # 开始递归调用
    recursive_rag_call(query)

---RETRIEVE---
"Node 'retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT USEFUL---
---GRADE: DOCUMENT USEFUL---
---GRADE: DOCUMENT USEFUL---
---GRADE: DOCUMENT USEFUL---
---GRADE: DOCUMENT USEFUL---
---GRADE: DOCUMENT USEFUL---
---GRADE: DOCUMENT USEFUL---
---ASSESS GRADED DOCUMENTS---
7
---DECISION: DISTILL AND GENERATE---
"Node 'grade_documents':"
'\n---\n'
---DISTILL---
"Node 'distill':"
'\n---\n'
---GENERATE---
Based on the provided context, the following properties can be specified:

1. **Non-Failure**: A monad `m` does not fail under precondition `P` if for no start state in that precondition it sets the failure flag.
   - `no_fail P m ≡ ∀ s. P s *−→ ¬* (snd (m s))`

2. **Validity**: A Hoare triple is valid if for all states that satisfy the precondition, all result values and result states that are returned by the monad satisfy the postcondition.
   - `valid P f Q ≡ ∀ s. P s −→ (∀ (r,s') ∈ fst (f s). Q r s')`

3. **Validity for Exception Monad**:

/tmp/ipykernel_604890/4217057816.py:20: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
/tmp/ipykernel_604890/4217057816.py:25: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(add_query_and_rel, query, sub_query)


"Node 'retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT USEFUL---
---GRADE: DOCUMENT USEFUL---
---GRADE: DOCUMENT USEFUL---
---GRADE: DOCUMENT USEFUL---
---GRADE: DOCUMENT NOT USEFUL---
---GRADE: DOCUMENT USEFUL---
---GRADE: DOCUMENT USEFUL---
---ASSESS GRADED DOCUMENTS---
6
---DECISION: DISTILL AND GENERATE---
"Node 'grade_documents':"
'\n---\n'
---DISTILL---
"Node 'distill':"
'\n---\n'
---GENERATE---
Based on the provided context, here are some properties that can be specified:

**Generic Functions on Top of the State Monad**

1. **Lifting a normal function into the monad type**: `liftM` should preserve the behavior of the original function, i.e., `liftM f m = m >>= return. f`.
2. **Lifting a function with an exception monad**: `liftME` should preserve the behavior of the original function, i.e., `liftME f m = m >>=E returnOk. f`.
3. **Sequencing monads**: `sequence_x` should execute the monads in the list from left to right, ignoring return values,

/tmp/ipykernel_604890/4217057816.py:20: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
/tmp/ipykernel_604890/4217057816.py:25: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(add_query_and_rel, query, sub_query)


"Node 'retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT USEFUL---
---GRADE: DOCUMENT NOT USEFUL---
---GRADE: DOCUMENT USEFUL---
---GRADE: DOCUMENT USEFUL---
---GRADE: DOCUMENT USEFUL---
---GRADE: DOCUMENT USEFUL---
---GRADE: DOCUMENT NOT USEFUL---
---ASSESS GRADED DOCUMENTS---
5
---DECISION: DISTILL AND GENERATE---
"Node 'grade_documents':"
'\n---\n'
---DISTILL---
"Node 'distill':"
'\n---\n'
---GENERATE---
Based on the provided context, the following properties can be specified for the Monad Laws:

1. **Left Identity**: `return x >>= f = f x`
   - This law states that applying `return` to a value and then binding it to a function is equivalent to directly applying the function to the value.

2. **Right Identity**: `m >>= return = m`
   - This law states that binding a monad to `return` does not change the monad.

3. **Associativity**: `(m >>= f) >>= g = m >>= (λx. f x >>= g)`
   - This law states that the order in which binds are performed does not af

...

/tmp/ipykernel_604890/4217057816.py:20: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
/tmp/ipykernel_604890/4217057816.py:25: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(add_query_and_rel, query, sub_query)


---GRADE: DOCUMENT USEFUL---
---GRADE: DOCUMENT USEFUL---
---GRADE: DOCUMENT USEFUL---
---GRADE: DOCUMENT NOT USEFUL---
---GRADE: DOCUMENT USEFUL---
---GRADE: DOCUMENT USEFUL---
---GRADE: DOCUMENT USEFUL---
---ASSESS GRADED DOCUMENTS---
6
---DECISION: DISTILL AND GENERATE---
"Node 'grade_documents':"
'\n---\n'
---DISTILL---
"Node 'distill':"
'\n---\n'
---GENERATE---
Based on the provided context, here are some properties that can be specified:

1. **Kernel Entry Point**: The kernel entry point, `call_kernel`, is the main function that handles events, preemption interrupts, scheduling, and thread activation.

Property: The kernel entry point should handle all events correctly and ensure that the system remains in a consistent state.

2. **System Call Handling**: The kernel provides various system calls, such as `Send`, `NBSend`, `Call`, `Wait`, `Reply`, and `ReplyWait`, which are handled by the `handle_invocation` function.

Property: Each system call should be handled correctly, and th

/tmp/ipykernel_604890/4217057816.py:20: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
/tmp/ipykernel_604890/4217057816.py:25: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(add_query_and_rel, query, sub_query)


---GRADE: DOCUMENT USEFUL---
---GRADE: DOCUMENT NOT USEFUL---
---GRADE: DOCUMENT USEFUL---
---GRADE: DOCUMENT USEFUL---
---GRADE: DOCUMENT USEFUL---
---GRADE: DOCUMENT USEFUL---
---GRADE: DOCUMENT NOT USEFUL---
---ASSESS GRADED DOCUMENTS---
5
---DECISION: DISTILL AND GENERATE---
"Node 'grade_documents':"
'\n---\n'
---DISTILL---
"Node 'distill':"
'\n---\n'
---GENERATE---
Based on the provided context, here are some properties that can be specified:

1. **Virtual Memory Rights**: The ARM-specific virtual memory rights can be specified as a set of properties that define the access permissions for virtual memory. For example:
	* `ParityEnabled`: a property that indicates whether parity checking is enabled for a virtual memory region.
	* `PageCacheable`: a property that indicates whether a virtual memory region is cacheable.
	* `Global`: a property that indicates whether a virtual memory region is global (i.e., accessible from all processes).
	* `XNever`: a property that indicates whether a

/tmp/ipykernel_604890/4217057816.py:20: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
/tmp/ipykernel_604890/4217057816.py:25: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(add_query_and_rel, query, sub_query)


In [24]:
leaf_nodes_with_parents[24:]

['Parent: seL4. Current Concept to specify: Error and Fault Messages',
 'Parent: seL4. Current Concept to specify: Access Rights',
 'Parent: ARM-Specific Data Types. Current Concept to specify: Architecture-specific virtual memory',
 'Parent: ARM-Specific Data Types. Current Concept to specify: Architecture-specific capabilities',
 'Parent: ARM-Specific Data Types. Current Concept to specify: Architecture-specific objects',
 'Parent: ARM-Specific Data Types. Current Concept to specify: Architecture-specific object types and default objects',
 'Parent: ARM-Specific Data Types. Current Concept to specify: Architecture-specific state',
 'Parent: Machine Operations. Current Concept to specify: Wrapping and Lifting Machine Operations',
 'Parent: Machine Operations. Current Concept to specify: The Operations',
 'Parent: Machine Operations. Current Concept to specify: Memory Clearance',
 'Parent: Machine Operations. Current Concept to specify: User Monad',
 'Parent: Basic Data Structures. Cur

In [7]:
import os
from neo4j import GraphDatabase

# 从环境变量加载Neo4j连接信息
NEO4J_URI = os.getenv("NEO4J_URI", "bolt://localhost:7687")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME", "neo4j")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD", "neo4jneo4j")

# 创建Neo4j驱动
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# 定义查询语句，获取与seL4节点通过SPECIFY关系直接连接的所有节点
query = """
MATCH (seL4 {name: "seL4"})-[:SPECIFY]->(relatedNode)
RETURN relatedNode.name AS name
"""

# 执行查询并获取结果
def get_specify_related_nodes(tx):
    result = tx.run(query)
    return [record["name"] for record in result]

with driver.session() as session:
    related_nodes = session.read_transaction(get_specify_related_nodes)
    print("Nodes directly connected to seL4 through SPECIFY:", related_nodes)

# 关闭驱动
driver.close()


Nodes directly connected to seL4 through SPECIFY: ['System Calls', 'An Initial Kernel State', 'Decoding System Calls', 'Kernel Invocation Labels', 'Interrupts', 'IPC', 'Threads and TCBs', 'Scheduler', 'Toplevel ARM Definitions', 'CSpace', 'Prefix order on lists as order class instance', 'IPC Cancelling', 'ARM VSpace Functions', 'Retyping and Untyped Invocations', 'Kernel Object Invocations', 'ARM Object Invocations', 'Accessing the ARM VSpace', 'Accessing CSpace', 'Accessing the Kernel Heap', 'Basic Kernel and Exception Monads', 'Abstract Specification Instantiations', 'Basic Data Structures', 'Machine Operations', 'ARM-Specific Data Types', 'ARM-Specific Virtual-Memory Rights', 'Access Rights', 'Error and Fault Messages', 'Machine Accessor Functions', 'ARM Machine Instantiation', 'Common, Architecture-Specific Data Types', 'Kernel Events', 'Machine Types', 'ARM Machine Types', 'Platform Definitions', 'Machine Word Setup', 'Enumeration instances for Words', 'Enumerations', 'Nondetermin

/tmp/ipykernel_7316/1393371063.py:24: DeprecationWarning: read_transaction has been renamed to execute_read
  related_nodes = session.read_transaction(get_specify_related_nodes)


In [16]:
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferWindowMemory
from langchain_community.chat_models import ChatOllama
from langchain.chains import ConversationChain, LLMChain

def formalize_specifications(context, json_tree):
    # 定义提示模板
    reduce_duplicates_prompt = '''
    In the formalization of operating systems, a specification is a precise, formal description of a system's behavior in code, serving as a blueprint for implementation and verification.
    Please eliminate any duplicate specifications from the following Specification Tree, which is a hierarchical structure where the leaf nodes represent individual specifications and the root node represents the overarching topic.
    Your response should strictly follow the format of the provided Specification Tree (each specification is in the "Name:Description" format), and return only the updated Specification Tree.
    Specification Tree:
    {specification}
    '''
    
    identify_purpose_prompt = '''
    The following code is a part of an Isabelle formalization. Isabelle is a proof assistant used to formalize mathematical proofs and system specifications.
    Based on the provided code, identify the primary purpose and intent of the formalization.
    Please consider the overall goal of the system or behavior being specified.
    Code:
    {code}
    '''
    
    identify_implemented_spec_prompt = '''
    Based on the following Specification Tree, identify the specifications that are already implemented in the provided code.
    Specification Tree:
    {specification}
    '''
    
    identify_missing_spec_prompt = '''
    What specifications are implemented in the code but are not listed in the Specification Tree? These will be referred to as code-specifications.
    Each code-specification should follow this format:
    Specification Name: A concise, one-sentence description.
    '''
    
    update_spec_tree_prompt = '''
    Please update the [Specification Tree] by removing duplicate specifications and adding the code-specifications in their appropriate places.
    Respond only with the final updated [Specification Tree], with no additional commentary.
    Ensure that the specifications are at the behavior level, not the function level.
    [Specification Tree to be updated]:
    {specification}
    '''
    
    # 初始化语言模型
    llm = ChatOllama(model='llama3.1:70b', temperature=0)
    
    # 创建记忆存储，限制为 2 轮对话
    memory = ConversationBufferWindowMemory(k=2)
    
    # 初始化对话链
    conversation = ConversationChain(
        llm=llm,
        memory=memory,
        verbose=True
    )
    
    # Step 1: 减少规约树的重复性
    reduce_duplicates_chain = LLMChain(
        llm=llm, 
        prompt=PromptTemplate(input_variables=["specification"], template=reduce_duplicates_prompt)
    )
    spec_tree_response = reduce_duplicates_chain.run(specification=json_tree)
    print("Reduced Specification Tree:", spec_tree_response)
    
    # Step 2: 识别Isabelle代码的目的和意图
    prompt_purpose = PromptTemplate(input_variables=["code"], template=identify_purpose_prompt)
    purpose_response = conversation.predict(input=prompt_purpose.format(code=context))
    print("Purpose Response:", purpose_response)
    
    # Step 3: 使用减少重复后的规约树识别已实现的规约
    prompt_identify_spec = PromptTemplate(input_variables=["specification"], template=identify_implemented_spec_prompt)
    implemented_spec_response = conversation.predict(input=prompt_identify_spec.format(specification=spec_tree_response))
    print("Implemented Specifications Response:", implemented_spec_response)
    
    # Step 4: 识别代码中未列出的规约
    prompt_missing_spec = PromptTemplate(input_variables=["specification", "code"], template=identify_missing_spec_prompt)
    missing_spec_response = conversation.predict(input=prompt_missing_spec.format(specification=implemented_spec_response, code=context))
    print("Missing Specifications Response:", missing_spec_response)
    
    # Step 5: 更新规约树
    prompt_update_tree = PromptTemplate(input_variables=["specification"], template=update_spec_tree_prompt)
    updated_spec_tree_response = conversation.predict(input=prompt_update_tree.format(specification=spec_tree_response))
    print("Updated Specification Tree Response:", updated_spec_tree_response)
    
    return updated_spec_tree_response


In [26]:
os.environ['HTTP_PROXY'] = ''
os.environ['HTTPS_PROXY'] = ''

In [ ]:
import os
import json
from neo4j import GraphDatabase
import glob

# Neo4j数据库连接信息
NEO4J_URI = os.getenv("NEO4J_URI", "bolt://localhost:7687")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME", "neo4j")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD", "neo4jneo4j")

# 创建数据库驱动
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

def get_tree(tx, root_node_name):
    query = """
    MATCH (n {name: $root_node_name})
    OPTIONAL MATCH (n)-[:SPECIFY]->(child)
    RETURN n.name AS name, collect(child.name) AS children
    """
    result = tx.run(query, root_node_name=root_node_name)
    record = result.single()
    if record:
        name = record["name"]
        children_names = [child for child in record["children"] if child is not None]
        children = []
        for child_name in children_names:
            child_tree = get_tree(tx, child_name)  # 递归获取子节点
            children.append(child_tree)
        return {"name": name, "children": children}
    else:
        return {"name": root_node_name, "children": []}  # 如果没有找到节点，返回空的子节点列表

def get_and_display_tree(root_node_name):
    with driver.session() as session:
        tree = session.read_transaction(get_tree, root_node_name)
        json_tree = json.dumps(tree, indent=2, ensure_ascii=False)
        return json_tree

os.chdir("/home/zhuxy/OSAutoFormalizer/KGConstruct/abstract")

for root_node_name in glob.glob("*.thy"):
    if root_node_name[:-4] in related_nodes:
        json_tree = get_and_display_tree(root_node_name[:-4])
        print(root_node_name[:-4])
        with open(root_node_name, "r", encoding="utf-8") as f:
            context = f.read()
        result = formalize_specifications(context, json_tree)
        with open('../result/qwen/'+root_node_name[:-4]+".json", "w", encoding="utf-8") as f:
            f.write(result)

os.chdir("../")

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferWindowMemory
from langchain_community.chat_models import ChatOllama
from langchain.chains import ConversationChain, LLMChain

# 定义提示模板

# Step 1: 减少规约树的重复性
reduce_duplicates_prompt = '''
In the formalization of operating systems, a specification is a precise, formal description of a system's behavior in code, serving as a blueprint for implementation and verification.
Please eliminate any duplicate specifications from the following Specification Tree, which is a hierarchical structure where the leaf nodes represent individual specifications and the root node represents the overarching topic.
Your response should strictly follow the format of the provided Specification Tree (each specification is in the "Name:Description" format), and return only the updated Specification Tree.
Specification Tree:
{specification}
'''

# Step 2: 识别Isabelle代码的目的和意图
identify_purpose_prompt = '''
The following code is a part of an Isabelle formalization. Isabelle is a proof assistant used to formalize mathematical proofs and system specifications.
Based on the provided code, identify the primary purpose and intent of the formalization.
Please consider the overall goal of the system or behavior being specified.
Code:
{code}
'''

# Step 3: 识别已实现的规约，使用减少重复后的规约树
identify_implemented_spec_prompt = '''
Based on the following Specification Tree, identify the specifications that are already implemented in the provided code.
Specification Tree:
{specification}
'''

# Step 4: 识别代码中未列出的规约
identify_missing_spec_prompt = '''
What specifications are implemented in the code but are not listed in the Specification Tree? These will be referred to as code-specifications.
Each code-specification should follow this format:
Specification Name: A concise, one-sentence description.
'''

# Step 5: 更新规约树
update_spec_tree_prompt = '''
Please update the [Specification Tree] by removing duplicate specifications and adding the code-specifications in their appropriate places.
Respond only with the final updated [Specification Tree], with no additional commentary.
Ensure that the specifications are at the behavior level, not the function level.
[Specification Tree to be updated]:
{specification}
'''

# 初始化语言模型
llm = ChatOllama(model='llama3.1:70b', temperature=0)

# 创建记忆存储，限制为 2 轮对话
memory = ConversationBufferWindowMemory(k=2)

# 初始化对话链
conversation = ConversationChain(
    llm=json_llm,
    memory=memory,
    verbose=True
)

# Step 1: 减少规约树的重复性
reduce_duplicates_chain = LLMChain(
    llm=json_llm, 
    prompt=PromptTemplate(input_variables=["specification"], template=reduce_duplicates_prompt)
)

# Step 2: 识别Isabelle代码的目的和意图
prompt_purpose = PromptTemplate(input_variables=["code"], template=identify_purpose_prompt)
purpose_response = conversation.predict(input=prompt_purpose.format(code=context))
print("Purpose Response:", purpose_response)

# Step 3: 使用减少重复后的规约树识别已实现的规约
spec_tree_response = reduce_duplicates_chain.run(specification=json_tree)
print("Reduced Specification Tree:", spec_tree_response)

prompt_identify_spec = PromptTemplate(input_variables=["specification"], template=identify_implemented_spec_prompt)
implemented_spec_response = conversation.predict(input=prompt_identify_spec.format(specification=spec_tree_response))
print("Implemented Specifications Response:", implemented_spec_response)

# Step 4: 识别代码中未列出的规约
prompt_missing_spec = PromptTemplate(input_variables=["specification", "code"], template=identify_missing_spec_prompt)
missing_spec_response = conversation.predict(input=prompt_missing_spec.format(specification=implemented_spec_response, code=context))
print("Missing Specifications Response:", missing_spec_response)

# Step 5: 更新规约树
prompt_update_tree = PromptTemplate(input_variables=["specification"], template=update_spec_tree_prompt)
updated_spec_tree_response = conversation.predict(input=prompt_update_tree.format(specification=spec_tree_response))
print("Updated Specification Tree Response:", updated_spec_tree_response)


In [69]:
def insert_tree(tx, tree, parent=None):

    node_name = tree["name"] 
    
    # 创建节点
    query = "CREATE (n:Specification {name: $name})"
    tx.run(query, name=node_name)
    
    # 如果有父节点，创建关系
    if parent:
        relation_query = """
        MATCH (parent:Specification {name: $parent_name}), (child:Specification {name: $child_name})
        CREATE (parent)-[:SPECIFY]->(child)
        """
        tx.run(relation_query, parent_name=parent, child_name=node_name)

    # 递归插入
    for child in tree.get("children", []):
        insert_tree(tx, child, node_name)

def insert_updated_spec_tree(updated_spec_tree_response):
    # 将响应转换为 JSON 格式
    tree = json.loads(updated_spec_tree_response)
    
    with driver.session() as session:
        session.write_transaction(insert_tree, tree)

# 插入更新后的规约树
insert_updated_spec_tree(updated_spec_tree_response)


/tmp/ipykernel_76421/2446247842.py:26: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(insert_tree, tree)


In [11]:
import os
from neo4j import GraphDatabase

# 从环境变量加载Neo4j连接信息
NEO4J_URI = os.getenv("NEO4J_URI", "bolt://localhost:7687")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME", "neo4j")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD", "neo4jneo4j")

# 创建Neo4j驱动
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# 定义查询语句，获取与seL4节点通过SPECIFY关系直接连接的所有节点
query = """
MATCH (seL4 {name: "seL4"})-[:SPECIFY]->(relatedNode)
RETURN relatedNode.name AS name
"""

# 执行查询并获取结果
def get_specify_related_nodes(tx):
    result = tx.run(query)
    return [record["name"] for record in result]

with driver.session() as session:
    related_nodes = session.read_transaction(get_specify_related_nodes)
    print("Nodes directly connected to seL4 through SPECIFY:", related_nodes)

# 关闭驱动
driver.close()
print(len(related_nodes))
for node in related_nodes:
    print(node)


Nodes directly connected to seL4 through SPECIFY: ['System Calls', 'An Initial Kernel State', 'Decoding System Calls', 'Kernel Invocation Labels', 'Interrupts', 'IPC', 'Threads and TCBs', 'Scheduler', 'Toplevel ARM Definitions', 'CSpace', 'Prefix order on lists as order class instance', 'IPC Cancelling', 'ARM VSpace Functions', 'Retyping and Untyped Invocations', 'Kernel Object Invocations', 'ARM Object Invocations', 'Accessing the ARM VSpace', 'Accessing CSpace', 'Accessing the Kernel Heap', 'Basic Kernel and Exception Monads', 'Abstract Specification Instantiations', 'Basic Data Structures', 'Machine Operations', 'ARM-Specific Data Types', 'ARM-Specific Virtual-Memory Rights', 'Access Rights', 'Error and Fault Messages', 'Machine Accessor Functions', 'ARM Machine Instantiation', 'Common, Architecture-Specific Data Types', 'Kernel Events', 'Machine Types', 'ARM Machine Types', 'Platform Definitions', 'Machine Word Setup', 'Enumeration instances for Words', 'Enumerations', 'Nondetermin

/tmp/ipykernel_7316/4166264086.py:24: DeprecationWarning: read_transaction has been renamed to execute_read
  related_nodes = session.read_transaction(get_specify_related_nodes)
